# Convert color images to RGB FITS

0. FITS images
1. Create color image, for example using [Trilogy](https://github.com/dancoe/trilogy)
2. Convert to RGB FITS (this notebook)
3. Compress using gzip (shrinks a lot!)
4. Load into ds9 / Aladin:

### Load RGB FITS image into ds9

* Scale -- Min Max
* Frame -- New Frame RGB
* File -- Open as -- RGB Image...

### Load RGB FITS image into APT Aladin

* Aladin
* Cmd-I
* File
* Browse
* RGB fits image
* (3 planes show up in Aladin)
* rgb
    * red = RGB[1]
    * green = RGB[2]
    * blue = RGB[3]

In [ ]:
import PIL
from PIL import Image, ImageEnhance
PIL.Image.MAX_IMAGE_PIXELS = 933120000  # avoid decompression bomb error when loading large images

import numpy as np
from astropy.io import fits

In [ ]:
def loadrgb(infile):
    im = Image.open(infile)
    im = im.transpose(method=Image.Transpose.FLIP_TOP_BOTTOM)
    rgb = np.asarray(im)  # numpy
    print(rgb.shape)
    rgb = np.transpose(rgb, (2,0,1))  # (ny, nx, 3) -> (3, ny, nx)
    rgb = rgb[:3]  # in case there's an alpha channel on the end
    rgb.flags.writeable = True  # DEFAULT IS CAN'T EDIT IT!
    return rgb

In [ ]:
def im2rgbfits(color_image_file, header_file, rgb_output_file='', overwrite=False, flip=False):
    if rgb_output_file == '':
        rgb_output_file = color_image_file[:-4] + '_rgb.fits'
        
    if os.path.exists(rgb_output_file):
        print(rgb_output_file, 'EXISTS')
        return
        
    rgb = loadrgb(color_image_file)
    header = fits.getheader(header_file)
    hdu = fits.PrimaryHDU(header=header)
    hdu_list = fits.HDUList([hdu])
    for i in range(3):
        hdu = fits.ImageHDU(rgb[i], header=header)
        hdu_list.append(hdu)
        
    hdu_list.writeto(output_file, overwrite=overwrite)

In [ ]:
color_image_file = 'whl0137.png'
input_fits_file = '../images/grizli_v2/20mas/sunrise-grizli-v2-f200w-clear_drc_sci.fits'
output_file = 'whl0137_rgb.fits'

In [ ]:
im2rgbfits(color_image_file, input_fits_file, output_file)